In [194]:
import csv
import pandas as pd
import numpy as np
import scipy.linalg as linalg


In [195]:
symptoms = ['A', 'B', 'C','D']

covid_df = pd.DataFrame(np.array([[3, 2, 2,5], [7, 8, 9, 0], [2, 2, 1,3]]), columns=symptoms)

In [196]:
print(covid_df.head())
# run this if you want to run this very quickly
# covid_df = covid_df.head(100)

   A  B  C  D
0  3  2  2  5
1  7  8  9  0
2  2  2  1  3


In [197]:
symptoms = ['A', 'B', 'C','D']
print(len(symptoms))

4


In [198]:
covid_df = covid_df.astype(float)
print(covid_df.head())


     A    B    C    D
0  3.0  2.0  2.0  5.0
1  7.0  8.0  9.0  0.0
2  2.0  2.0  1.0  3.0


## 2. Graph Building
9 nodes

In [199]:
edges = []
for i in range(len(symptoms)):
        for j in range(i +1, len(symptoms)):
            edges.append((i,j))
triangles = []
for i in range(len(symptoms)):
        for j in range(i +1, len(symptoms)):
            for h in range(j + 1, len(symptoms)):
                triangles.append((i,j, h))
adj_matrix = np.zeros((len(symptoms), len(symptoms)))
curl = np.zeros((len(edges), len(triangles)))
neg_divergence = np.zeros((len(edges), len(symptoms)))
f = np.zeros((len(edges)))
W = np.zeros((len(edges), len(edges)))

In [200]:
print(triangles)
print(edges)

[(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [201]:
# f, w
for index, row in covid_df.iterrows():
    for i, edge in enumerate(edges):
        if (row[symptoms[edge[0]]] != 0) and (row[symptoms[edge[1]]] != 0):
            f[i] += -((row[symptoms[edge[0]]] - row[symptoms[edge[1]]]))
            W[i, i] += 1
for i in range(len(edges)):
    f[i] = f[i]*1/W[i,i]

In [202]:
print(f)
print(W)

[0.  0.  1.5 0.  2.  2.5]
[[3. 0. 0. 0. 0. 0.]
 [0. 3. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 2.]]


In [203]:
# adj_matrix
for index, row in covid_df.iterrows():
    for i in range(len(symptoms)):
        for j in range(i +1, len(symptoms)):
            adj_matrix[i, j] += (row[symptoms[i]] - row[symptoms[j]])/len(covid_df) #divide by #ppl with both symptoms
print(adj_matrix)

[[0.         0.         0.         1.33333333]
 [0.         0.         0.         1.33333333]
 [0.         0.         0.         1.33333333]
 [0.         0.         0.         0.        ]]


In [204]:
# neg_divergence
for i in range(len(edges)):
    for j in range(len(symptoms)):
        if edges[i][0] == j:
            neg_divergence[i,j] = -1
        elif edges[i][1] == j:
            neg_divergence[i,j] = 1
print(neg_divergence)

[[-1.  1.  0.  0.]
 [-1.  0.  1.  0.]
 [-1.  0.  0.  1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  1.]
 [ 0.  0. -1.  1.]]


In [205]:
print(np.matmul(neg_divergence, np.transpose(neg_divergence)))

[[ 2.  1.  1. -1. -1.  0.]
 [ 1.  2.  1.  1.  0. -1.]
 [ 1.  1.  2.  0.  1.  1.]
 [-1.  1.  0.  2.  1. -1.]
 [-1.  0.  1.  1.  2.  1.]
 [ 0. -1.  1. -1.  1.  2.]]


In [206]:
# curl
for j, tri in enumerate(triangles):
    for i, edge in enumerate(edges):
        if edge[0] in tri and edge[1] in tri:
            first_edge = tri.index(edge[0])
            if (first_edge + 1) % 3 == tri.index(edge[1]):
                curl[i, j] = 1
            else:
                curl[i,j] = -1
print(curl)

[[ 1.  1.  0.  0.]
 [-1.  0.  1.  0.]
 [ 0. -1. -1.  0.]
 [ 1.  0.  0.  1.]
 [ 0.  1.  0. -1.]
 [ 0.  0.  1.  1.]]


### Solving for r

In [207]:
right_side = np.matmul(np.transpose(neg_divergence), np.matmul(W, f))
left_side = np.matmul(np.matmul(np.transpose(neg_divergence), W), neg_divergence)
r = np.matmul(linalg.pinv(left_side), right_side)
print(r)

[-0.40909091 -0.5        -0.59090909  1.5       ]


In [212]:
right_side = np.matmul(np.transpose(neg_divergence),  f)
left_side = np.matmul(np.transpose(neg_divergence), neg_divergence)
r = np.matmul(linalg.pinv(left_side), right_side)
print(r)

[-0.375 -0.5   -0.625  1.5  ]


In [214]:
r = [4, 4, 4, 4]

In [215]:
sum = 0
#(r[edges[i][0]] - r[edges[i][1]])
for i in range(len(edges)):
    to_add = W[i, i]*(f[i] + (r[edges[i][0]] - r[edges[i][1]]))*(f[i] + (r[edges[i][0]] - r[edges[i][1]]))
    
    sum += to_add
    print(to_add)
print(sum)

0.0
0.0
4.5
0.0
8.0
12.5
25.0


In [165]:
sum = 0
#(r[edges[i][0]] - r[edges[i][1]])
for i in range(len(edges)):
    sum += W[i, i]*(f[i] - (r[edges[i][0]] - r[edges[i][1]]))*(f[i] - (r[edges[i][0]] - r[edges[i][1]]))
print(sum)

97.54545454545453


### Solving for c

In [74]:
right_side = np.matmul(np.transpose(curl), np.matmul(W, f))
left_side = np.matmul(np.matmul(np.transpose(curl), W), curl)
c = np.matmul(np.linalg.inv(left_side), right_side)
print(c)

[ 15728640. -13107200.  -7733248.   6733824.   2359296.   -983040.
  -1966080.  26214400.   6815744.  -8847360. -10485760.   3145728.
   1572864.  -1572864.   6160384.  -7602176.  12582912.   2228224.
   8912896.  -3014656. -12058624.   -393216.  14155776.   4587520.
  -5242880.   7667712. -11796480.  12582912.  -4194304.  -4194304.
  -4718592.   8486912. -18087936.  11272192.   3276800.  14155776.
  -4980736.   1015808.  -7864320.  -5242880.   7340032.   2064384.
   3145728.    786432.  -2883584.  -1081344.   1048576.   5931008.
 -14680064.   8912896. -12910592.   2916352. -12058624.    -65536.
   6553600.  -7864320.   7208960.   3276800.   7864320.    -65536.
  -1064960.  -5636096.   9961472.   5767168.  -1605632.   6160384.
 -10485760.  -2162688.  -9437184.    278528.  -1572864.   8912896.
   -983040.  -2031616. -24117248.   6553600.   8388608. -17301504.
  -4980736.   -499712.   5636096.   3145728.  -4063232.   1966080.]
